In [6]:
'''
Re-implementation for paper "Classification of hyperspectral remote sensing images with support vector machines"
The official implementation is in https://github.com/mhaut/hyperspectral_deeplearning_review
'''

############ IMPORTS ####################
import sys
sys.path.append("../")
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import torch
from operator import truediv
import record
from sklearn import svm
import pickle

############# CONFIGS ##########################

# os.environ["CUDA_VISIBLE_DEVICES"]="1"

datasetNames = ["Trento"]
patchsize1 = 11
patchsize2 = 11
batchsize = 128
EPOCH = 4
LR = 0.001


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100
        
for datasetName in datasetNames:
    print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
    TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
    TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    
    # average pooling in dimension 2 and 3
    TrainPatch1 = torch.nn.functional.adaptive_avg_pool2d(TrainPatch1,1)
    TestPatch1 = torch.nn.functional.adaptive_avg_pool2d(TestPatch1,1)
    
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
        
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((1, Classes))

    for iter in range(1):
        clf = svm.SVC(gamma=0.03125, C=1e5, tol=1e-7)
        valdata =  (TestPatch1.reshape(TestPatch1.shape[0],-1), TestLabel1.reshape(-1))
        clf.fit(TrainPatch1.reshape(TrainPatch1.shape[0],-1), TrainLabel1.reshape(-1))
        pickle.dump(clf, open(datasetName+"/best_model_HSIOnly.h5", 'wb'))
        clf = pickle.load(open(datasetName+"/best_model_HSIOnly.h5", 'rb'))
        pred_y = clf.predict(TestPatch1.reshape(TestPatch1.shape[0],-1))
        
        

        y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
        oa = accuracy_score(y_test, pred_y)*100
        confusion = confusion_matrix(y_test, pred_y)
        each_acc, aa = AA_andEachClassAccuracy(confusion)
        kappa = cohen_kappa_score(y_test, pred_y)*100
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iter, :] = each_acc
    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/SVM_Report_' + datasetName +'.txt')


----------------------------------Training for  Trento ---------------------------------------------
Train data shape =  torch.Size([819, 63, 1, 1])
Train label shape =  torch.Size([1, 819])
Test data shape =  torch.Size([29395, 63, 1, 1])
Test label shape =  torch.Size([1, 29395])
--------Trento Training Finished-----------
